In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pycountry
from sklearn.preprocessing import MultiLabelBinarizer, LabelBinarizer
pd.set_option('display.max_columns', 500)
random_state = 42

In [2]:
stack_overflow_files = (os.listdir("data/"))
# not worth going from 2011-2014. No data scientists.
# ok, so decision to do (2019 maybe) 2020-2023 for analysis

# GPT gave me this idea instead of going through every possible country manually
country_abbreviations_1 = {country.name: country.alpha_3 for country in pycountry.countries}
country_abbreviations_2 = {country.official_name: country.alpha_3 for country in pycountry.countries}
os.listdir("data/")

C:\Users\luism\anaconda3\envs\milestone_1\Lib\site-packages\pycountry\db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)


['ppp.csv', 'salaries.csv', 'stack_overflow']

## Functions

In [3]:
def create_onehot_skills(frames: dict) -> dict:
    """
    Given a dictionary of pandas dataframes we want to one hot the skills in particular.
    We want to take the skills in the different columns and one hot them such we can sum them for groupby operations.
    We get a dictionary of pandas DataFrames and perform an inplace operation such that we don't have to create new memory.
    Return a dictionary of a list of strings for a couple reasons:
        - there's no way we will remember all of these so automation by putting these into a list seemed like the best idea
        - the keys will match those in the input in case we want to do something with these later per year
        - hashing onto a dictionary should allow for ease of access since no 2 years will have the same EXACT one hot columns, hence the list

    We also drop the _Empty for EVERYTHING since that information is useless to us
    
    Input: frames dict{str: pd.DataFrames}
    Ouput: dict{str: list[str]}

    https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list

    Rough example flow of function for one sample:
    C; C++; Perl -> [C, C++, Perl] -> [1, 1, 1, 0]
    Python       -> [Python]       -> [0, 0, 0, 1]
    """
    # some constants
    standard = [("language", "lg"), ("database", "db"), ("platform", "pf"), ("webframe", "wf"), ("misctech", "mt")]
    status = [("wanttoworkwith", "www"), ("haveworkedwith", "hww")]

    new_cols_per_year = {}
    
    for key, frame in frames.items():
        new_cols = []
        for stan, abv in standard:
            for stat, abr in status:
                coi = stan + stat # coi = column of interest
                abbr = abv + abr + "_"
                mlb = MultiLabelBinarizer(sparse_output=True) # saves ram
                frame[coi] = frame[coi].str.split(";")
                transformed = mlb.fit_transform(frame.pop(coi))
                new_cois = [abbr + name for name in mlb.classes_]
                frame = frame.join(
                            pd.DataFrame.sparse.from_spmatrix(
                                transformed,
                                index=frame.index,
                                columns=new_cois
                            )
                        )
                new_cois.remove(abbr + "Empty")
                new_cols += new_cois
                frame = frame.drop(abbr + "Empty", axis=1)
        # this needs to be here, if not throse Sparse type errors
        # # Sparse types don't allow normal groupby operations (ie reshape) so we need to turn them into ints
        # # int8 don't take up a ton and it's just 0's and 1's
        # # for all intents and purposes these are sparse matrices, we just want to avoid the object
        frame[new_cols] = frame[new_cols].fillna(0)
        frame[new_cols] = frame[new_cols].astype('int8')
        frames[key] = frame
        new_cols_per_year[key] = new_cols
    return new_cols_per_year

In [4]:
def abbr_education(frames: dict) -> None:
    """
    Similar in spirit to the other one hots, but this is in place
    Automatically abbreviates education levels across all frames
    Had to hard code the list again, not a big deal only 8 items
    
    Input: frames dict{str: pd.DataFrames}
    Ouput: None
    """
    # more hardcoded stuff that are needed
    abbreviations = ["ad", "bs", "ms", "ele", "prof", "sec", "scu", "else"]
    
    for key, frame in frames.items():
        # easier to replace this, makes it much easier to work with
        frame['edlevel'] = frame['edlevel'].replace({'I never completed any formal education': 'Something else'})

        # need the sorted since they have the same rough scheme
        levels = list(frame['edlevel'].unique())
        levels.sort()
        o = 0 # offset

        # dictionary to feed into repalce function
        replace_dict = {}
        for i in range(len(levels)):
            col = levels[i]
            if col == 'nan':
                break
            abbr = abbreviations[i-o]
            if 'doctoral' in col:
                replace_dict[col] = "phd"
                o += 1
                continue
            replace_dict[col] = abbr
                
        frame['edlevel'] = frame['edlevel'].replace(replace_dict)
        frames[key] = frame

In [5]:
def create_onehot_edu(frames: dict) -> None:
    """
    Inplace one hotting of frames dictionary for edulevel column.
    We are dropping nan's again since we aren't able to guess an education level.
    They don't give us any information and would only be an obstacle in analysis.
    
    Input: frames dict{str: pd.DataFrame}
    Output: None
    """
    # inplace
    abbr_education(frames)

    # go through our frames
    for key, frame in frames.items():
        # mlb doesn't work since it will use char, and we want str
        lb = LabelBinarizer(sparse_output=True) # saves ram
        transformed = lb.fit_transform(frame.pop('edlevel'))
        frame = frame.join(
                    pd.DataFrame.sparse.from_spmatrix(
                        transformed,
                        index=frame.index,
                        columns=lb.classes_
                    )
                )
        new_cols = list(lb.classes_)

        # edge case so all columns line up
        if 'phd' not in frame.columns:
            frame['phd'] = [0] * len(frame)
            new_cols.append('phd')
        
        # tells us nothing
        if 'nan' in frame.columns:
            frame = frame.drop('nan', axis=1)
            new_cols.remove('nan')
        frame[new_cols] = frame[new_cols].fillna(0)
        frame[new_cols] = frame[new_cols].astype('int8')
        frames[key] = frame

In [6]:
def read_stackoverflow() -> (dict, dict):
    """
    Reads CSVs and gets the numbe of data professionals. Any empty values are dropped from job title and 
    salary so we will always have data. Other columns may have nans.
    Data Manipulation:
    - dropping nans from salary and devtype combined
    - Changing the salary column to ConvertedCompYearly so we can merge all data frames comes time
    - Lowering column names since there was some weird camel case going on
    - Converting specific columns that mean the same thing per year into a singular name
    - Fill in nans for language/skill specific values with "Empty"
      - this is so we can one hot later on for a more concise analysis, more later on
    - One hot encoding the different skills per year, see create_onehot_skills
    - Next we abbreviate education levels so that we can also one hot them, see above
    - One hot education, see other documentation

    Inputs: Nothing
    Outputs: tuple(dict{str: pd.DataFrame}, dict{str: list[str]})
    """
    frames = {}
    stack_o_files = os.listdir("data/stack_overflow/")
    for file in stack_o_files:
        year = file[-8:-4]
        df = pd.read_csv(f"data/stack_overflow/{file}", encoding='ISO-8859-1')

        # standardize compensation columns
        if 'ConvertedComp' in df.columns:
            df = df.rename(columns={'ConvertedComp': 'ConvertedCompYearly'})

        # standardize some columns
        # using camel case resulted in errors with webframe where sometimes F was capitalized
        standard = ["language", "database", "platform", "webframe", "misctech"]
        df.columns = df.columns.str.lower()
        for stan in standard:
            if f"{stan}workedwith" in df.columns:
                df = df.rename(columns={f'{stan}workedwith': f'{stan}haveworkedwith', f'{stan}desirenextyear':f'{stan}wanttoworkwith'})
            df[f"{stan}haveworkedwith"] = df[f"{stan}haveworkedwith"].fillna(value="Empty")
            df[f"{stan}wanttoworkwith"] = df[f"{stan}wanttoworkwith"].fillna(value="Empty")

        # standardize some country names, now they should match with Kaggle dataset
        df["country"] = df["country"].replace(country_abbreviations_1)
        df["country"] = df["country"].replace(country_abbreviations_2)

        df['edlevel'] = df['edlevel'].fillna(value="nan")
        
        df = df.dropna(subset=["devtype", "convertedcompyearly"])
        df = df[df["devtype"].str.contains("data", case=False)]
        df["count"] = [1] * len(df) # this is for our groupby so that we can say count > cull when we sum or count
        df["year"] = [year] * len(df)
        frames[f"df_data_{year}"] = df

    # oops forgot indentation
    create_onehot_edu(frames)
    new_cols = create_onehot_skills(frames)
    return frames, new_cols

## The Data

In [7]:
frames_dict, new_cols = read_stackoverflow()

In [8]:
# df = frames_dict["df_data_2019"]

In [9]:
# df.dtypes[df. dtypes == 'Sparse[int32, 0]']

In [10]:
# list(frames_dict["df_data_2019"].dtypes)

In [11]:
# this is the number of entries we are working with in our frames
# seeing how to standardize the columns some more
# this is kind of useless now with one hotting everything

query = "Web"
for key, frame in frames_dict.items():
    lang = []
    for col in frame.columns:
        lang.append(col) if query in col else None
    print(f"{key}\t{len(frame)}\t{lang}")

df_data_2019	13393	['lgwww_WebAssembly', 'lghww_WebAssembly']
df_data_2020	8294	[]
df_data_2021	9272	[]
df_data_2022	6921	[]
df_data_2023	2480	['pfwww_Amazon Web Services (AWS)', 'pfhww_Amazon Web Services (AWS)']


In [12]:
# print(frames_dict["df_data_2019"].columns)

In [13]:
# print(frames_dict["df_data_2020"].columns)

In [14]:
# print(frames_dict["df_data_2021"].columns)

In [15]:
# print(frames_dict["df_data_2022"].columns)

In [16]:
# print(frames_dict["df_data_2023"].columns)

## Similarity with columns per the dataframes

In [17]:
# do they have similar columns?
def find_similar_col(frames) -> set:
    """
    Returns the set of columns that the all share, ideally we maximize the ratio of this to merge.
    """
    union = []
    for key, frame in frames.items():
        union.append(set(frame.columns))
        
    standard = union[0]
    for cols in union[1:]:
        standard = standard.intersection(cols)
    return standard

In [18]:
find_similar_col(frames_dict)

{'ad',
 'age',
 'bs',
 'comptotal',
 'convertedcompyearly',
 'count',
 'country',
 'dbhww_Cassandra',
 'dbhww_Couchbase',
 'dbhww_Elasticsearch',
 'dbhww_MariaDB',
 'dbhww_Microsoft SQL Server',
 'dbhww_MongoDB',
 'dbhww_MySQL',
 'dbhww_Oracle',
 'dbhww_PostgreSQL',
 'dbhww_Redis',
 'dbhww_SQLite',
 'dbwww_Cassandra',
 'dbwww_Couchbase',
 'dbwww_Elasticsearch',
 'dbwww_MariaDB',
 'dbwww_Microsoft SQL Server',
 'dbwww_MongoDB',
 'dbwww_MySQL',
 'dbwww_Oracle',
 'dbwww_PostgreSQL',
 'dbwww_Redis',
 'dbwww_SQLite',
 'devtype',
 'ele',
 'else',
 'employment',
 'lghww_Assembly',
 'lghww_C',
 'lghww_C#',
 'lghww_C++',
 'lghww_Dart',
 'lghww_Go',
 'lghww_HTML/CSS',
 'lghww_Java',
 'lghww_JavaScript',
 'lghww_Kotlin',
 'lghww_Objective-C',
 'lghww_PHP',
 'lghww_Python',
 'lghww_R',
 'lghww_Ruby',
 'lghww_Rust',
 'lghww_SQL',
 'lghww_Scala',
 'lghww_Swift',
 'lghww_TypeScript',
 'lghww_VBA',
 'lgwww_Assembly',
 'lgwww_C',
 'lgwww_C#',
 'lgwww_C++',
 'lgwww_Dart',
 'lgwww_Go',
 'lgwww_HTML/CSS',

## Countries given a cull factor

In [19]:
# play around with the number and see if this is the spread that we want
for key, frame in frames_dict.items():
    print(key)
    grouped = frame.groupby("country").count()
    grouped = grouped[grouped["mainbranch"] > 10]
    length = len(grouped)
    print(f"""{key}: {length}
    max: {grouped['mainbranch'].idxmax()}, {grouped['mainbranch'].max()}
    min: {grouped['mainbranch'].idxmin()}, {grouped['mainbranch'].min()}""")

df_data_2019
df_data_2019: 83
    max: USA, 3856
    min: ARM, 11
df_data_2020
df_data_2020: 69
    max: USA, 2081
    min: BLR, 11
df_data_2021
df_data_2021: 70
    max: USA, 2144
    min: BIH, 11
df_data_2022
df_data_2022: 63
    max: USA, 1702
    min: EGY, 11
df_data_2023
df_data_2023: 36
    max: USA, 687
    min: CHN, 11


In [20]:
# do they have similar columns?
def find_similar_country(frames: dict, cull_factor=20) -> set:
    """
    Given a particular minimum (cull_factor) find the countries in common among
    frames.
    """
    union = []
    for key, frame in frames.items():
        grouped = frame.groupby("country").count()
        grouped = grouped[grouped["mainbranch"] > cull_factor]
        union.append(set(grouped.index))
        
    standard = union[0]
    for cols in union[1:]:
        standard = standard.intersection(cols)
    return standard

def show_country_dist(frames: dict, countries: list, cull_factor: int) -> None:
    """
    Just plot a bar chart for our country distributions using the above function.
    """
    rows = len(frames)//2 + 1
    fig, axes = plt.subplots(nrows=rows, ncols=2, figsize=(15,15))
    fig.suptitle(f"{len(countries)} respondents consistent across surveys greater than {cull_factor} responses")
    for (key, frame), ax in zip(frames.items(), axes.reshape(-1)):
        grouped = frame.groupby("country").count()
        grouped = grouped.loc[list(countries)].sort_values("mainbranch")
        grouped.plot(y="mainbranch", ax=ax, kind="bar", legend=False)
        ax.set_title(key[-4:])
    
    plt.show()

In [21]:
# across all data sets here are the countries that are here most often
# where is US? UK? They have different, inconsistent names throughout the years
# # i.e. United States vs United States of America; UK vs United Kingdom, see above mapping
cull_factor = 20
country_sim = find_similar_country(frames_dict, cull_factor)
# show_country_dist(frames_dict, list(country_sim), cull_factor)

## One Hot Testing for Skills (deprecated)

In [22]:
# basically with every one of these is separated by a ;
# goal of next function:
# # find the sub-strings separated by ; nans will have to be replaced by "None" or "Empty"
# # one hot the entries for example, if C appears in one of these queries, for that particular
# # subject there will be a 1 for yes and 0 for no essentially
# # this is why we need the None/Empty so we can add them up
# # Eventually after one hotting we drop the None/Empty since it's a dummy column
# # we would then be able to add them up using count or something and put onto a graph/analysis


# standard = ["language", "database", "platform", "webframe", "misctech"]
# want = "wanttoworkwith"
# have = "haveworkedwith"
# for key, frame in frames_dict.items():
#     print(key)
#     for stan in standard:
#         print(f"{stan}: {frame[stan + want].isna().sum()} {frame[stan + want].sample(n=1).values}")
#     print()

In [23]:
# df = frames_dict["df_data_2019"].copy(deep=True) # don't want this to point at the frame in dict

In [24]:
# coi = 'languagewanttoworkwith'
# df[coi] = df[coi].str.split(";")
# mlb = MultiLabelBinarizer(sparse_output=True) # saves ram

# transformed = mlb.fit_transform(df.pop(coi))
# columns = ["langwork_" + name for name in mlb.classes_]

# df = df.join(
#             pd.DataFrame.sparse.from_spmatrix(
#                 transformed,
#                 index=df.index,
#                 columns=columns))

In [25]:
# frame = frames_dict["df_data_2019"].copy(deep=True)

In [26]:
# df.groupby('country').sum()[mlb.classes_]

In [27]:
# def create_onehot_skills(frames: dict):
#     # some constants
#     standard = [("language", "lg"), ("database", "db"), ("platform", "pf"), ("webframe", "wf"), ("misctech", "mt")]
#     status = [("wanttoworkwith", "www"), ("haveworkedwith", "hww")]

#     new_cols_per_year = {}
    
#     for key, frame in frames.items():
#         new_cols = []
#         print(key)
#         for stan, abv in standard:
#             for stat, abr in status:
#                 coi = stan + stat # coi = column of interest
#                 abbr = abv + abr + "_"
#                 mlb = MultiLabelBinarizer(sparse_output=True) # saves ram
#                 frame[coi] = frame[coi].str.split(";")
#                 transformed = mlb.fit_transform(frame.pop(coi))
#                 new_cois = [abbr + name for name in mlb.classes_]
#                 frame = frame.join(
#                             pd.DataFrame.sparse.from_spmatrix(
#                                 transformed,
#                                 index=frame.index,
#                                 columns=new_cois
#                             )
#                         )
#                 new_cois.remove(abbr + "Empty")
#                 new_cols += new_cois
#                 frame.drop(abbr + "Empty", axis=1)
#         frames[key] = frame
#         new_cols_per_year[key] = new_cols
#     return new_cols_per_year

In [28]:
# import copy
# cp_dict = copy.deepcopy(frames_dict)

In [29]:
# new_cols = create_onehot_skills(cp_dict)

## Ed Level Processing (deprecated)

In [30]:
# one-hot education for same reason
# same thing
# we have nans and doctoral degrees missing from 2023 

In [31]:
# import copy
# cp_dict = copy.deepcopy(frames_dict)
# abbr_education(cp_dict)

In [32]:
# for key, frame in cp_dict.items():
#     frame['edlevel'] = frame['edlevel'].replace({'I never completed any formal education': 'Something else'})
    
#     do = list(frame['edlevel'].unique())
#     print(key, len(do))
#     do.sort()
#     display(do)
#     print()

In [33]:
# for key, frame in cp_dict.items():
#     lb = LabelBinarizer(sparse_output=True) # saves ram
#     transformed = lb.fit_transform(frame.pop('edlevel'))
#     frame = frame.join(
#                 pd.DataFrame.sparse.from_spmatrix(
#                     transformed,
#                     index=frame.index,
#                     columns=lb.classes_
#                 )
#             )
#     if 'phd' not in frame.columns:
#         frame['phd'] = [0] * len(frame)
#     print(frame.columns[-10:])

## Employment

In [34]:
find_similar_col(frames_dict)

{'ad',
 'age',
 'bs',
 'comptotal',
 'convertedcompyearly',
 'count',
 'country',
 'dbhww_Cassandra',
 'dbhww_Couchbase',
 'dbhww_Elasticsearch',
 'dbhww_MariaDB',
 'dbhww_Microsoft SQL Server',
 'dbhww_MongoDB',
 'dbhww_MySQL',
 'dbhww_Oracle',
 'dbhww_PostgreSQL',
 'dbhww_Redis',
 'dbhww_SQLite',
 'dbwww_Cassandra',
 'dbwww_Couchbase',
 'dbwww_Elasticsearch',
 'dbwww_MariaDB',
 'dbwww_Microsoft SQL Server',
 'dbwww_MongoDB',
 'dbwww_MySQL',
 'dbwww_Oracle',
 'dbwww_PostgreSQL',
 'dbwww_Redis',
 'dbwww_SQLite',
 'devtype',
 'ele',
 'else',
 'employment',
 'lghww_Assembly',
 'lghww_C',
 'lghww_C#',
 'lghww_C++',
 'lghww_Dart',
 'lghww_Go',
 'lghww_HTML/CSS',
 'lghww_Java',
 'lghww_JavaScript',
 'lghww_Kotlin',
 'lghww_Objective-C',
 'lghww_PHP',
 'lghww_Python',
 'lghww_R',
 'lghww_Ruby',
 'lghww_Rust',
 'lghww_SQL',
 'lghww_Scala',
 'lghww_Swift',
 'lghww_TypeScript',
 'lghww_VBA',
 'lgwww_Assembly',
 'lgwww_C',
 'lgwww_C#',
 'lgwww_C++',
 'lgwww_Dart',
 'lgwww_Go',
 'lgwww_HTML/CSS',

In [35]:
year = 2023
col = "orgsize"
print(frames_dict[f"df_data_{year}"][col].unique())
print(frames_dict[f"df_data_{year}"][col].isna().sum())

['500 to 999 employees'
 'Just me - I am a freelancer, sole proprietor, etc.'
 '100 to 499 employees' '1,000 to 4,999 employees' '10 to 19 employees'
 '2 to 9 employees' '20 to 99 employees' 'I donâ\x80\x99t know'
 '10,000 or more employees' '5,000 to 9,999 employees']
0


In [36]:
# print(list(frames_dict["df_data_2023"].columns))

In [37]:
df = frames_dict["df_data_2023"]

In [41]:
grouped = df.groupby('country').agg({"count":["sum"], "convertedcompyearly":["mean", "std"]})

In [43]:
grouped.columns

MultiIndex([(              'count',  'sum'),
            ('convertedcompyearly', 'mean'),
            ('convertedcompyearly',  'std')],
           )

In [45]:
# this is how we would cull, not awful but also not best thing in the world
grouped = grouped[grouped[("count", "sum")] > cull_factor]
grouped

count convertedcompyearly               
          sum                mean            std
country                                         
AUS        76        93834.868421   50171.824672
AUT        25       119040.720000  199982.206651
BEL        36        79305.388889   35446.929717
BRA        72        58933.819444  201131.648916
CAN        87       108793.034483  127513.909569
CHE        28       140053.464286   60025.678845
DEU       185        89562.378378   47131.465787
DNK        34       101817.705882   44357.573418
ESP        76        59160.315789   48200.329645
FIN        22        74806.045455   19511.362632
FRA       107        65571.000000   30233.700918
GBR       206        87087.703883   49364.479534
IND        95        27374.663158   21624.291013
ISR        28       128074.535714   41777.512216
ITA        49        47969.571429   23082.326163
NLD        72        89339.402778   47763.828288
NZL        21        86196.952381   35795.098087
POL        60        64081.483333   42061.947198
PRT        21        49248.666667   32333.119047
SWE        35       102438.971429  163351.435846
USA       687       179245.924309  357064.100561